# 天池o2o优惠券使用预测比赛解析（初级）

**赛题链接：**

[天池o2o优惠券使用预测](https://tianchi.aliyun.com/getStart/introduction.htm?spm=5176.100066.0.0.518433afBqXIKM&raceId=231593)

## 导入相关库

In [1]:
# import libraries necessary for this project
import os, sys, pickle
 
import numpy as np
import pandas as pd
 
from datetime import date
 
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler
 
# display for this notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 导入数据

In [2]:
dfoff = pd.read_csv('data/ccf_offline_stage1_train.csv',keep_default_na=False)
dfon = pd.read_csv('data/ccf_online_stage1_train.csv',keep_default_na=False)
dftest = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',keep_default_na=False)
 
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,null,null,0,null,20160217
1,1439408,4663,11002,150:20,1,20160528,null
2,1439408,2632,8591,20:1,0,20160217,null
3,1439408,2632,1078,20:1,0,20160319,null
4,1439408,2632,8591,20:1,0,20160613,null


In [42]:
dftest.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,discount_type,discount_rate,discount_man,discount_jian,distance,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,4129537,450,9983,30:5,1,20160712,1,0.833333,30,5,1,2,0,0,1,0,0,0,0,0
1,6949378,1300,3429,30:5,null,20160706,1,0.833333,30,5,-1,3,0,0,0,1,0,0,0,0
2,2166529,7113,6928,200:20,5,20160727,1,0.900000,200,20,5,3,0,0,0,1,0,0,0,0
3,2166529,7113,1808,100:10,5,20160727,1,0.900000,100,10,5,3,0,0,0,1,0,0,0,0
4,6172162,7605,6500,30:1,2,20160708,1,0.966667,30,1,2,5,0,0,0,0,0,1,0,0


In [43]:
dfon.head(5)

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,13740231,18907,2,100017492,500:50,20160513,null
1,13740231,34805,1,null,null,null,20160321
2,14336199,18907,0,null,null,null,20160618
3,14336199,18907,0,null,null,null,20160618
4,14336199,18907,0,null,null,null,20160618


In [21]:
aa = dfoff
aa['flag_coupon'] = np.where(aa['Date_received'] == 'null','no','yes')
aa['flag_deal'] = np.where(aa['Date'] == 'null','no','yes')
bb = pd.pivot_table(aa,index=["flag_coupon","flag_deal"],values=["User_id"],aggfunc=np.size)
bb = bb.reset_index()
bb

,flag_coupon,flag_deal,User_id
0,no,yes,701602
1,yes,no,977900
2,yes,yes,75382


### 简单统计

In [22]:
aa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754884 entries, 0 to 1754883
Data columns (total 9 columns):
User_id          int64
Merchant_id      int64
Coupon_id        object
Discount_rate    object
Distance         object
Date_received    object
Date             object
flag_coupon      object
flag_deal        object
dtypes: int64(2), object(7)
memory usage: 120.5+ MB


简单统计一下用户使用优惠券的情况：

In [24]:
print('有优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'] != 'null') & (dfoff['Date'] != 'null')].shape[0])
print('有优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'] != 'null') & (dfoff['Date'] == 'null')].shape[0])
print('无优惠卷，购买商品：%d' % dfoff[(dfoff['Date_received'] == 'null') & (dfoff['Date'] != 'null')].shape[0])
print('无优惠卷，未购商品：%d' % dfoff[(dfoff['Date_received'] == 'null') & (dfoff['Date'] == 'null')].shape[0])

有优惠卷，购买商品：75382
有优惠卷，未购商品：977900
无优惠卷，购买商品：701602
无优惠卷，未购商品：0


可见，很多人（701602）购买商品却没有使用优惠券，也有很多人（977900）有优惠券但却没有使用，真正使用优惠券购买商品的人（75382）很少！所以，这个比赛的意义就是把优惠券送给真正可能会购买商品的人。

## 特征提取

In [25]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,flag_coupon,flag_deal
0,1439408,2632,null,null,0,null,20160217,no,yes
1,1439408,4663,11002,150:20,1,20160528,null,yes,no
2,1439408,2632,8591,20:1,0,20160217,null,yes,no
3,1439408,2632,1078,20:1,0,20160319,null,yes,no
4,1439408,2632,8591,20:1,0,20160613,null,yes,no


In [27]:
dfoff['Discount_rate'].value_counts()

null       701602
30:5       270712
100:10     182554
200:20     111046
20:5        91013
20:1        51705
50:5        47379
100:30      38196
200:30      29327
300:30      28979
50:10       28452
10:5        25925
0.95        20568
10:1        17842
30:1        17654
150:20      17437
100:20      14297
30:10       12692
50:20        8203
0.9          8085
200:50       5585
150:10       5325
100:5        5053
0.8          3441
50:1         3354
5:1          2526
100:50       1774
150:30        654
0.85          649
200:10        575
100:1         537
20:10         514
150:50        306
300:50        206
0.5           186
0.75          121
0.2           110
0.6            58
200:5          57
300:20         56
0.7            54
30:20          24
300:10         23
200:100        12
50:30           9
150:5           7
Name: Discount_rate, dtype: int64

### 打折率 Discount_rate

In [26]:
print('Discount_rate 类型：\n',dfoff['Discount_rate'].unique())

Discount_rate 类型：
 ['null' '150:20' '20:1' '200:20' '30:5' '50:10' '10:5' '100:10' '200:30'
 '20:5' '30:10' '50:5' '150:10' '100:30' '200:50' '100:50' '300:30'
 '50:20' '0.9' '10:1' '30:1' '0.95' '100:5' '5:1' '100:20' '0.8' '50:1'
 '200:10' '300:20' '100:1' '150:30' '300:50' '20:10' '0.85' '0.6' '150:50'
 '0.75' '0.5' '200:5' '0.7' '30:20' '300:10' '0.2' '50:30' '200:100'
 '150:5']


打折率分为 3 种情况：

- 'null' 表示没有打折

- [0,1] 表示折扣率

- x:y 表示满x减y

**处理方式：**

- 打折类型：getDiscountType()

- 折扣率：convertRate()

- 满多少：getDiscountMan()

- 减多少：getDiscountJian()

In [28]:
# Convert Discount_rate and Distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
    """Convert discount to rate"""
    if row == 'null':
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)
    
def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0
    
def processData(df):
    
    # convert discount_rate
    df['discount_type'] = df['Discount_rate'].apply(getDiscountType)
    df['discount_rate'] = df['Discount_rate'].apply(convertRate)
    df['discount_man'] = df['Discount_rate'].apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].apply(getDiscountJian)
    
    print(df['discount_rate'].unique())
    
    return df

In [29]:
dfoff = processData(dfoff)
dftest = processData(dftest)

[1.         0.86666667 0.95       0.9        0.83333333 0.8
 0.5        0.85       0.75       0.66666667 0.93333333 0.7
 0.6        0.96666667 0.98       0.99       0.975      0.33333333
 0.2        0.4       ]
[0.83333333 0.9        0.96666667 0.8        0.95       0.75
 0.98       0.5        0.86666667 0.6        0.66666667 0.7
 0.85       0.33333333 0.94       0.93333333 0.975      0.99      ]


In [30]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,flag_coupon,flag_deal,discount_type,discount_rate,discount_man,discount_jian
0,1439408,2632,null,null,0,null,20160217,no,yes,null,1.000000,0,0
1,1439408,4663,11002,150:20,1,20160528,null,yes,no,1,0.866667,150,20
2,1439408,2632,8591,20:1,0,20160217,null,yes,no,1,0.950000,20,1
3,1439408,2632,1078,20:1,0,20160319,null,yes,no,1,0.950000,20,1
4,1439408,2632,8591,20:1,0,20160613,null,yes,no,1,0.950000,20,1


### 距离 Distance

In [31]:
print('Distance 类型：',dfoff['Distance'].unique())

Distance 类型： ['0' '1' 'null' '2' '10' '4' '7' '9' '3' '5' '6' '8']


将距离 str 转为 int。

In [32]:
# convert distance
dfoff['distance'] = dfoff['Distance'].replace('null', -1).astype(int)
print(dfoff['distance'].unique())
dftest['distance'] = dftest['Distance'].replace('null', -1).astype(int)
print(dftest['distance'].unique())

[ 0  1 -1  2 10  4  7  9  3  5  6  8]
[ 1 -1  5  2  0 10  3  6  7  4  9  8]


In [33]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,flag_coupon,flag_deal,discount_type,discount_rate,discount_man,discount_jian,distance
0,1439408,2632,null,null,0,null,20160217,no,yes,null,1.000000,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,yes,no,1,0.866667,150,20,1
2,1439408,2632,8591,20:1,0,20160217,null,yes,no,1,0.950000,20,1,0
3,1439408,2632,1078,20:1,0,20160319,null,yes,no,1,0.950000,20,1,0
4,1439408,2632,8591,20:1,0,20160613,null,yes,no,1,0.950000,20,1,0


### 领劵日期 Date_received

In [34]:
date_received = dfoff['Date_received'].unique()
date_received = sorted(date_received[date_received != 'null'])

date_buy = dfoff['Date'].unique()
date_buy = sorted(date_buy[date_buy != 'null'])

print('优惠卷收到日期从',date_received[0],'到',date_received[-1])
print('消费日期从',date_buy[0],'到',date_buy[-1])

优惠卷收到日期从 20160101 到 20160615
消费日期从 20160101 到 20160630


In [36]:
date_buy[0]

'20160101'

**关于领劵日期的特征：**

- weekday : {null, 1, 2, 3, 4, 5, 6, 7}

- weekday_type : {1, 0}（周六和周日为1，其他为0）

- Weekday_1 : {1, 0, 0, 0, 0, 0, 0}

- Weekday_2 : {0, 1, 0, 0, 0, 0, 0}

- Weekday_3 : {0, 0, 1, 0, 0, 0, 0}

- Weekday_4 : {0, 0, 0, 1, 0, 0, 0}

- Weekday_5 : {0, 0, 0, 0, 1, 0, 0}

- Weekday_6 : {0, 0, 0, 0, 0, 1, 0}

- Weekday_7 : {0, 0, 0, 0, 0, 0, 1}

In [37]:
def getWeekday(row):
    if row == 'null':
        return row
    else:
        return date(int(row[0:4]), int(row[4:6]), int(row[6:8])).weekday() + 1

dfoff['weekday'] = dfoff['Date_received'].astype(str).apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].astype(str).apply(getWeekday)

# weekday_type :  周六和周日为1，其他为0
dfoff['weekday_type'] = dfoff['weekday'].apply(lambda x: 1 if x in [6,7] else 0)
dftest['weekday_type'] = dftest['weekday'].apply(lambda x: 1 if x in [6,7] else 0)

# change weekday to one-hot encoding 
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
#print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace('null', np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace('null', np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

In [38]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,flag_coupon,flag_deal,discount_type,...,distance,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7
0,1439408,2632,null,null,0,null,20160217,no,yes,null,...,0,null,0,0,0,0,0,0,0,0
1,1439408,4663,11002,150:20,1,20160528,null,yes,no,1,...,1,6,1,0,0,0,0,0,1,0
2,1439408,2632,8591,20:1,0,20160217,null,yes,no,1,...,0,3,0,0,0,1,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,yes,no,1,...,0,6,1,0,0,0,0,0,1,0
4,1439408,2632,8591,20:1,0,20160613,null,yes,no,1,...,0,1,0,1,0,0,0,0,0,0


### 所有特征：

- discount_rate

- discount_type

- discount_man

- discount_jian

- distance

- weekday

- weekday_type

- weekday_1

- weekday_2

- weekday_3

- weekday_4

- weekday_5

- weekday_6

- weekday_7

## 标签标注

三种情况：

- Date_received == 'null'：表示没有领到优惠券，无需考虑，y = -1

- (Date_received != 'null') & (Date != 'null') & (Date - Date_received <= 15)：表示领取优惠券且在15天内使用，即正样本，y = 1

- (Date_received != 'null') & ((Date == 'null') | (Date - Date_received > 15))：表示领取优惠券未在在15天内使用，即负样本，y = 0

定义标签备注函数：

In [39]:
def label(row):
    if row['Date_received'] == 'null':
        return -1
    if row['Date'] != 'null':
        td = pd.to_datetime(row['Date'], format='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff['label'] = dfoff.apply(label, axis=1)

In [40]:
print(dfoff['label'].value_counts())

 0    988887
-1    701602
 1     64395
Name: label, dtype: int64


In [41]:
dfoff.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,flag_coupon,flag_deal,discount_type,...,weekday,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label
0,1439408,2632,null,null,0,null,20160217,no,yes,null,...,null,0,0,0,0,0,0,0,0,-1
1,1439408,4663,11002,150:20,1,20160528,null,yes,no,1,...,6,1,0,0,0,0,0,1,0,0
2,1439408,2632,8591,20:1,0,20160217,null,yes,no,1,...,3,0,0,0,1,0,0,0,0,0
3,1439408,2632,1078,20:1,0,20160319,null,yes,no,1,...,6,1,0,0,0,0,0,1,0,0
4,1439408,2632,8591,20:1,0,20160613,null,yes,no,1,...,1,0,1,0,0,0,0,0,0,0


## 建立线性模型 SGDClassifier

- 使用上面提取的14个特征。

- 训练集：20160101-20160515；验证集：20160516-20160615。

- 用线性模型 SGDClassifier

### 划分训练集/验证集

In [44]:
# data split
df = dfoff[dfoff['label'] != -1].copy()
train = df[(df['Date_received'] < '20160516')].copy()
valid = df[(df['Date_received'] >= '20160516') & (df['Date_received'] <= '20160615')].copy()
print('Train Set: \n', train['label'].value_counts())
print('Valid Set: \n', valid['label'].value_counts())

Train Set: 
 0    759172
1     41524
Name: label, dtype: int64
Valid Set: 
 0    229715
1     22871
Name: label, dtype: int64


### 特征数量

In [45]:
# feature
original_feature = ['discount_rate','discount_type','discount_man', 'discount_jian','distance', 'weekday', 'weekday_type'] + weekdaycols
print('共有特征：',len(original_feature),'个')
print(original_feature)

共有特征： 14 个
['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'distance', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


### 建立模型

In [46]:
def check_model(data, predictors):
    
    classifier = lambda: SGDClassifier(
        loss='log',  # loss function: logistic regression
        penalty='elasticnet', # L1 & L2
        fit_intercept=True,  # 是否存在截距，默认存在
        max_iter=100, 
        shuffle=True,  # Whether or not the training data should be shuffled after each epoch
        n_jobs=1, # The number of processors to use
        class_weight=None) # Weights associated with classes. If not given, all classes are supposed to have weight one.
 
    # 管道机制使得参数集在新数据集（比如测试集）上的重复使用，管道机制实现了对全部步骤的流式化封装和管理。
    model = Pipeline(steps=[
        ('ss', StandardScaler()), # transformer
        ('en', classifier())  # estimator
    ])
 
    parameters = {
        'en__alpha': [ 0.001, 0.01, 0.1],
        'en__l1_ratio': [ 0.001, 0.01, 0.1]
    }
 
    # StratifiedKFold用法类似Kfold，但是他是分层采样，确保训练集，测试集中各类别样本的比例与原始数据集中相同。
    folder = StratifiedKFold(n_splits=3, shuffle=True)
    
    # Exhaustive search over specified parameter values for an estimator.
    grid_search = GridSearchCV(
        model, 
        parameters, 
        cv=folder, 
        n_jobs=-1,  # -1 means using all processors
        verbose=1)
    grid_search = grid_search.fit(data[predictors], 
                                  data['label'])
    
    return grid_search

### 训练

In [47]:
predictors = original_feature
model = check_model(train, predictors)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:   53.5s finished


### 验证

对验证集中每个优惠券预测的结果计算 AUC，再对所有优惠券的 AUC 求平均。计算 AUC 的时候，如果 label 只有一类，就直接跳过，因为 AUC 无法计算。

In [48]:
# valid predict
y_valid_pred = model.predict_proba(valid[predictors])
valid1 = valid.copy()
valid1['pred_prob'] = y_valid_pred[:, 1]
valid1.head(5)

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,flag_coupon,flag_deal,discount_type,...,weekday_type,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,label,pred_prob
1,1439408,4663,11002,150:20,1,20160528,null,yes,no,1,...,1,0,0,0,0,0,1,0,0,0.020416
4,1439408,2632,8591,20:1,0,20160613,null,yes,no,1,...,0,1,0,0,0,0,0,0,0,0.094253
6,1439408,2632,8591,20:1,0,20160516,20160613,yes,yes,1,...,0,1,0,0,0,0,0,0,0,0.094253
9,2029232,450,1532,30:5,0,20160530,null,yes,no,1,...,0,1,0,0,0,0,0,0,0,0.091876
10,2029232,6459,12737,20:1,0,20160519,null,yes,no,1,...,0,0,0,0,1,0,0,0,0,0.130454


计算 AUC：

In [49]:
# avgAUC calculation
vg = valid1.groupby(['Coupon_id'])
aucs = []
for i in vg:
    tmpdf = i[1] 
    if len(tmpdf['label'].unique()) != 2:
        continue
    fpr, tpr, thresholds = roc_curve(tmpdf['label'], tmpdf['pred_prob'], pos_label=1)
    aucs.append(auc(fpr, tpr))
print(np.average(aucs))

0.5299519833666739


### 测试

In [50]:
# test prediction for submission
y_test_pred = model.predict_proba(dftest[predictors])
dftest1 = dftest[['User_id','Coupon_id','Date_received']].copy()
dftest1['Probability'] = y_test_pred[:,1]
dftest1.to_csv('submit1.csv', index=False, header=False)
dftest1.head(5)

,User_id,Coupon_id,Date_received,Probability
0,4129537,9983,20160712,0.104001
1,6949378,3429,20160706,0.152259
2,2166529,6928,20160727,0.005410
3,2166529,1808,20160727,0.018298
4,6172162,6500,20160708,0.064588


### 保存模型 & 导入模型

In [51]:
if not os.path.isfile('1_model.pkl'):
    with open('1_model.pkl', 'wb') as f:
        pickle.dump(model, f)
else:
    with open('1_model.pkl', 'rb') as f:
        model = pickle.load(f)

/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator StandardScaler from version 0.19.0 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator SGDClassifier from version 0.19.0 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version 0.19.0 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.19.0 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWa

## 优化模型...

- **特征工程**

- **机器学习算法**

- **模型集成**

## 参考代码

[比赛第一名代码与解析](https://github.com/wepe/O2O-Coupon-Usage-Forecast)